##Setup

In [1]:
!pip install torch transformers bitsandbytes accelerate sqlparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 36.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

15835660288


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [6]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [7]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE teachers (
    teacher_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    subject_taught VARCHAR(100),
    publications_count INTEGER
);

CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    teacher_id INTEGER,
    FOREIGN KEY (teacher_id) REFERENCES teachers(teacher_id)
);

CREATE TABLE publications (
    publication_id INTEGER PRIMARY KEY,
    title VARCHAR(200),
    author_id INTEGER,
    publication_date DATE,
    FOREIGN KEY (author_id) REFERENCES teachers(teacher_id)
);

CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    title VARCHAR(200),
    description TEXT,
    instructor_id INTEGER,
    FOREIGN KEY (instructor_id) REFERENCES teachers(teacher_id)
);

CREATE TABLE events (
    event_id INTEGER PRIMARY KEY,
    title VARCHAR(200),
    date DATE,
    location VARCHAR(100),
    description TEXT
);

CREATE TABLE research_domains (
    domain_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    description TEXT
);

- Faculty: Details about faculty members, including their ID, name, subject expertise, and publication record.
  - Students: Information on enrolled students, including their ID, name, and associated faculty advisor.
  - Publications: Records of scholarly publications authored by faculty, with details such as publication ID, title, author ID, and publication date.
  - Courses: Descriptions of courses offered by ISFCR, including course ID, title, description, and instructor details.
  - Events: Listings of upcoming events hosted or endorsed by ISFCR, featuring event ID, title, date, venue, and description.
  - Research Areas: Categories delineating the various research domains within ISFCR, with attributes including domain ID, name, and description.
- Establish appropriate relationships between tables using foreign keys to maintain data integrity.

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [8]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [9]:
combined_list = [
    # Faculty Related
    "What are the names of all the faculty members?",
    "How many publications does each faculty member have?",
    "Which faculty member teaches the course titled 'Introduction to Cybersecurity'?",

    # Students Related
    "Who are the enrolled students?",
    "Which students are supervised by a specific faculty member?",
    "How many students are associated with each faculty member?",

    # Publications Related
    "What are the titles of all publications in the database?",
    "Who are the authors of the publication titled 'Advanced Cryptography'?",
    "When was the publication titled 'Digital Forensics Handbook' published?",

    # Courses Related
    "What are the titles of all available courses?",
    "Who is the instructor for the course with ID 101?",
    "Provide a description of the course titled 'Network Security Fundamentals'.",

    # Events Related
    "What events are scheduled for next month?",
    "Where will the event titled 'Cybersecurity Conference 2024' take place?",
    "Provide a brief description of the event with ID 201.",

    # Research Domains Related
    "What are the research domains covered by ISFCR?",
    "Describe the research domain with ID 301.",
    "Which faculty members are associated with the research domain 'Digital Forensics'?"
]

for i in combined_list:
  generated_sql = generate_query(i)
  print(generated_sql)


SELECT t.name
FROM teachers t;

SELECT t.name,
       COUNT(p.publication_id) AS publication_count
FROM teachers t
JOIN publications p ON t.teacher_id = p.author_id
GROUP BY t.name
ORDER BY publication_count DESC NULLS LAST;

SELECT t.name
FROM teachers t
JOIN courses c ON t.teacher_id = c.instructor_id
WHERE c.title ilike '%Introduction%to%Cybersecurity%'

SELECT s.name
FROM students s;

SELECT s.student_id,
       s.name
FROM students s
WHERE s.teacher_id = 757071;

SELECT t.name,
       COUNT(s.student_id) AS number_of_students
FROM teachers t
JOIN students s ON t.teacher_id = s.teacher_id
GROUP BY t.name
ORDER BY number_of_students DESC NULLS LAST;

SELECT p.title
FROM publications p;

SELECT p.author_id
FROM publications p
WHERE p.title ilike '%Advanced%Cryptography%'

SELECT p.publication_date
FROM publications p
WHERE p.title ilike '%Digital%Forensics%Handbook%'

SELECT c.title
FROM courses c;

SELECT c.instructor_id
FROM courses c
WHERE c.course_id = 101;

SELECT c.description

Tasks:

1)Generate a sizable database and present intricate queries necessitating multiple joins. Experiment with other state-of-the-art models in the same field
to accomplish this task.

2) This lab showcased employing a Transformer Model to generate SQL commands from natural language. It's conceivable to automate the generation of data through a Transformer model and feed it into a SQL creator. Subsequently, utilize the same Transformer model to retrieve the content stored in the database.